In [1]:
# Bank id

b_id = 29
#bc_ids = [17, 9, 29, 26, 50, 4, 5, 13, 64]

In [2]:
import warnings

warnings.filterwarnings("error")

In [3]:
import sys

sys.path.append('../..')

In [4]:
def rec_dict_print(a_dict, level=1):
    keys = a_dict.keys()
    keys.sort()
    for k in keys:
        if type(a_dict[k]) is dict:
            print "".join(["-" for i in range(level)]) + k
            rec_dict_print(a_dict[k], level + 2)
        else:
            print "".join(["-" for i in range(level)]) + k

In [5]:
%matplotlib inline
import dill

loaded_data = None

with open('../precomputed_sims/data.pkl', 'rb') as f:
    loaded_data = dill.load(f)

In [6]:
rec_dict_print(loaded_data)

-N
-credit
---bc_ids
---bc_subsets_indexes
---copula
---default_times
-randomization
---distrib
-----loc
-----scale
---maturity
---zetas
-underlyings


In [7]:
# Instead of taking 365 standard days or 252 trading days
# in order to get some easy computations for the eqty and df time grids
# I chose to take 360 days of tradings

step = 1/360.
delta = 15*step

maturity = loaded_data["randomization"]["maturity"]

print "Maturity = %s years" % maturity

Maturity = 5.0 years


# Discount

In [8]:
from finance.discountfactor import ConstantRateDiscountFactor 

r = 0.02
discount = ConstantRateDiscountFactor(r)

# Underlying

In [9]:
udlyings = loaded_data["underlyings"]

print "Maximum number of paths: %i" % len(udlyings)

gbm0 = udlyings[0]

kappa = gbm0.drifts[0][0]
sigma = gbm0.vols[0][0]
print "kappa = %s, sigma = %s" % (kappa, sigma)

time_grid = gbm0.time

Maximum number of paths: 20000
kappa = 0.12, sigma = 0.2


# Derivative

In [10]:
derivatives_nb = 1

In [11]:
from finance.products.european.swap import (
    SwapContract,
)

swap_delta = 0.25

swap_dates = SwapContract.generate_payment_dates(0, maturity, swap_delta)
swap = SwapContract(gbm0, discount, swap_dates)

price_0 = swap.price(0., incl_next_coupon=True)

print swap
print "\nPrice swap at t=0 with 1st coupon = %s" % price_0

Swap contract of maturity T = 5 years, over S^0 with strike K = 134.306, paying at {0.00, 0.25, 0.50, 0.75, 1.00, 1.25, 1.50, 1.75, 2.00, 2.25, 2.50, 2.75, 3.00, 3.25, 3.50, 3.75, 4.00, 4.25, 4.50, 4.75, 5.00}

Price swap at t=0 with 1st coupon = 0.0


# Exposure

In [12]:
from risk.exposures import EuropeanVaRGeomBrownianExposure

quantile_im = 0.80

exposure = EuropeanVaRGeomBrownianExposure(swap, discount, kappa, sigma)

In [13]:
from risk.basel.eee import BlackScholesSwapVaREffExpectExposure

eee = BlackScholesSwapVaREffExpectExposure(swap)

# Indexes stuffs

## Ids of $B$ and $C$

In [14]:
copula = loaded_data["credit"]["copula"]

bc_subsets_indexes = loaded_data["credit"]["bc_subsets_indexes"]

#bc_ids = [26, 29, 17, 50, 13, 4, 5, 9, 64]
#bc_positions = [-0.36, -0.44, 0.69, 0.34, -0.05, 0.23, 0.09, -0.46, -0.04]
#Spreads [1053, 367, 176, 73, 61, 56, 52, 45, 108]

bc_ids = [17, 9, 29, 26, 50, 4, 5, 13, 64]
bc_positions = [0.69, -0.46, -0.44, -0.36, 0.34, 0.23, 0.09, -0.05, -0.04]

# Counterparties id
c_ids = list(set(bc_ids) - set([b_id]))

print "Bank id: %s" % b_id
print "Counterparties id: %s (nb = %s)" % (c_ids, len(c_ids))

Bank id: 29
Counterparties id: [64, 4, 5, 9, 13, 17, 50, 26] (nb = 8)


In [15]:
obligors_nb = len(copula.subsets[bc_subsets_indexes[-1]][0])

print "Obligor numbers: %s" % obligors_nb

Obligor numbers: 125


## Subsets indexes of the copula for $B$ and $C$

In [16]:
b_subsets_indexes = copula.get_indexes_including(b_id)

c_subsets_indexes = []
for c_id in c_ids:
    tmp = copula.get_indexes_including(c_id)
    c_subsets_indexes += tmp
    
b_subsets_indexes.sort()
c_subsets_indexes.sort()
    
print "Subsets that generated default times: %s\n" % bc_subsets_indexes
print "Bank subsets indexes: %s\n" % b_subsets_indexes

tmp_c_subsets_indexes = list(set(c_subsets_indexes))
tmp_c_subsets_indexes.sort()
print "Counterparties subsets indexes: %s" % tmp_c_subsets_indexes

Subsets that generated default times: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129]

Bank subsets indexes: [29, 126, 127, 128, 129]

Counterparties subsets indexes: [4, 5, 9, 13, 17, 26, 50, 64, 125, 126, 127, 128, 129]


# Portfolio construction

In [17]:
import numpy as np

p_fixed = 1.
strike = swap.strike

delta_times = swap.delta_time
discount_factors = [discount(t) for t in swap.pillars[1:]]

delta_beta_sum = np.dot(delta_times, discount_factors)

notional = p_fixed / (strike*delta_beta_sum)

print "Notional = %s" % notional

Notional = 0.0015687485053


In [18]:
pos = np.zeros(obligors_nb)
for idx, ps in zip(bc_ids, bc_positions):
    pos[idx] = ps
    
positions = pos.reshape(pos.size, 1)

print pos

[ 0.    0.    0.    0.    0.23  0.09  0.    0.    0.   -0.46  0.    0.    0.
 -0.05  0.    0.    0.    0.69  0.    0.    0.    0.    0.    0.    0.    0.
 -0.36  0.    0.   -0.44  0.    0.    0.    0.    0.    0.    0.    0.    0.
  0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.34
  0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
 -0.04  0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
  0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
  0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
  0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
  0.    0.    0.    0.    0.    0.    0.    0.    0.  ]


In [19]:
from finance.portfolio import CSAPortfolio

port = CSAPortfolio(positions, [notional], [swap], [exposure], b_id)

# VM and IM accounts

In [20]:
from ccp.accounts import Accounts
from ccp.states import MembersState

states = MembersState(obligors_nb)
vm_accounts = Accounts(states, derivatives_nb)
im_accounts = Accounts(states, derivatives_nb)

In [21]:
from credit.default_models import StepwiseConstantIntensity

default_proba_models = []

all_ids = list(copula.subsets[bc_subsets_indexes[-1]][0])
for ii in all_ids:
    _model = None
    if ii in c_ids:
        _indexes = copula.get_indexes_including(ii)
        mat_hzrd_rate = dict()
        for m in copula.models[_indexes]:
            for pill, hzrd_rate in zip(m.pillars, m.intensities):
                mat_hzrd_rate[pill] = mat_hzrd_rate.get(pill, 0.) + hzrd_rate
                #if pill not in mat_hzrd_rate:
                #    mat_hzrd_rate[pill] = 0.
                #mat_hzrd_rate[pill] += hzrd_rate
                
        _model = StepwiseConstantIntensity(mat_hzrd_rate.keys(), mat_hzrd_rate.values())
    
    default_proba_models.append(_model)
    
print default_proba_models

[None, None, None, None, <credit.default_models.StepwiseConstantIntensity object at 0x000000002E5015C0>, <credit.default_models.StepwiseConstantIntensity object at 0x000000002E501B70>, None, None, None, <credit.default_models.StepwiseConstantIntensity object at 0x0000000071F9BB70>, None, None, None, <credit.default_models.StepwiseConstantIntensity object at 0x0000000071F9BBA8>, None, None, None, <credit.default_models.StepwiseConstantIntensity object at 0x0000000071F9B208>, None, None, None, None, None, None, None, None, <credit.default_models.StepwiseConstantIntensity object at 0x0000000071F9BCF8>, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, <credit.default_models.StepwiseConstantIntensity object at 0x0000000071F9BD68>, None, None, None, None, None, None, None, None, None, None, None, None, None, <credit.default_models.StepwiseConstantIntensity object at 0x0000000071F9BF60>, None, None, None,

# Regulatory capital

In [22]:
from risk.basel.exposures import BaselExposureAtDefault

ead = BaselExposureAtDefault(port, [eee])

In [23]:
from risk.basel.capital import CSARegulatoryCapital

recoveries = [0.4 for _ in all_ids]
regul_cap = CSARegulatoryCapital(ead, states, default_proba_models, recoveries)
epsilon_ead = 1./12
cap_conf_level = quantile_im

# Funding and capital parameters

In [24]:
lambda_ = 0.
funder_recovery = 1.
k_ = 0.1
c_ = 0.

# MC loop

In [25]:
from utils import time_offseter

In [26]:
N = loaded_data["N"]
N = 10000

In [27]:
results_globs = {"cva": {"sum": 0., "sum2": 0.},
                 "dva": {"sum": 0., "sum2": 0.},
                 "mla": {"sum": 0., "sum2": 0.},
                 "mva": {"sum": 0., "sum2": 0.},
                 "kva": {"sum": 0., "sum2": 0.}}

In [28]:
from scipy.stats import expon

exp_distrib_params = loaded_data["randomization"]["distrib"]
exp_distrib = expon(loc=exp_distrib_params["loc"], scale=exp_distrib_params["scale"])

zetas = loaded_data["randomization"]["zetas"]
default_times_mat = loaded_data["credit"]["default_times"]

In [29]:
import time

tic = time.time()

b_df_times_indexes = [ii for ii, ind in enumerate(bc_subsets_indexes) if ind in b_subsets_indexes]
mat_delta = maturity + delta

for i in range(N):
    # Resurrecting the states
    # resets to 0. the values
    # of the VMs and IMs
    states.resurrect_all()
    port.delete_cache()
    
    zeta = zetas[i]
    zeta_delta = time_offseter(zeta+delta, time_grid, True)
    
    if zeta <= maturity + delta:
        discount_zeta = discount(zeta)
        
        b_gamma_zeta = copula.tot_gamma(zeta, subsets_indexes=b_subsets_indexes)
        bar_lambda_zeta = b_gamma_zeta * 0.6
        tilde_lambda_zeta = bar_lambda_zeta - (1 - funder_recovery) * b_gamma_zeta
        lambda_zeta = 0.
        
        c_zeta = c_
        k_zeta = k_
        inv_pdf_zeta = 1./exp_distrib.pdf(zeta)
    
    # We set the simulated underlying here
    # that has the all time grid with
    # mpor = 1./360
    swap.underlying = udlyings[i]
    
    default_times = default_times_mat[i]
    b_min_df_time = default_times[b_df_times_indexes].min()
    
    cva_, dva_, fva_, kva_ = [0. for _ in range(4)]
    
    for c_id in c_ids:
        cp_subsets_indexes = copula.get_indexes_including(c_id)
        c_df_times_indexes = [ii for ii, ind in enumerate(bc_subsets_indexes) if ind in cp_subsets_indexes]
        
        c_min_df_time = default_times[c_df_times_indexes].min()
        bc_min_df_time = min(b_min_df_time, c_min_df_time)
        if bc_min_df_time < maturity and c_min_df_time <= b_min_df_time + delta:
            time_payoff = min(b_min_df_time, c_min_df_time) + delta
            time_payoff = time_offseter(time_payoff, time_grid, True)
            
            lgd = 1.-recoveries[c_id]
            
            ###########
            ### Q^i ###
            ###########            
            p_i = port.compute_value(time_payoff, from_=c_id, towards_=b_id).sum()
            q_i = p_i
            
            ############
            ### VM^i ###
            ############
            vm_i = port.compute_value(bc_min_df_time, from_=c_id, towards_=b_id)
            vm_accounts.put_amounts(b_id, -vm_i)
            vm_accounts.put_amounts(c_id, vm_i)
            
            #################
            ### IM^{c, i} ###
            #################
            im_bi = port.compute_exposure(bc_min_df_time, 
                                          risk_period=delta, 
                                          conf_level=quantile_im,
                                          from_=c_id, 
                                          towards_=b_id)
            
            im_ci = -port.compute_exposure(bc_min_df_time, 
                                          risk_period=delta, 
                                          conf_level=quantile_im,
                                          from_=b_id, 
                                          towards_=c_id)
            
            im_accounts.put_amounts(b_id, im_bi)
            im_accounts.put_amounts(c_id, -im_ci)
            
            c_ci = vm_i.sum() + im_ci.sum()
            
            #############
            ### CVA^i ###
            #############
            cva_ += - discount(time_payoff) * lgd * np.minimum(q_i - c_ci, 0.)
        
        if bc_min_df_time < maturity and b_min_df_time <= c_min_df_time + delta:
            time_payoff = min(b_min_df_time, c_min_df_time) + delta
            time_payoff = time_offseter(time_payoff, time_grid, True)
            
            lgd = 1.-recoveries[b_id]
            
            ###########
            ### Q^i ###
            ###########            
            p_i = port.compute_value(time_payoff, from_=c_id, towards_=b_id).sum()
            q_i = p_i
            
            ############
            ### VM^i ###
            ############
            vm_i = port.compute_value(bc_min_df_time, from_=c_id, towards_=b_id)
            vm_accounts.put_amounts(b_id, -vm_i)
            vm_accounts.put_amounts(c_id, vm_i)
            
            #################
            ### IM^{c, i} ###
            #################
            im_bi = port.compute_exposure(bc_min_df_time, 
                                          risk_period=delta, 
                                          conf_level=quantile_im,
                                          from_=c_id, 
                                          towards_=b_id)
            
            im_ci = -port.compute_exposure(bc_min_df_time, 
                                          risk_period=delta, 
                                          conf_level=quantile_im,
                                          from_=b_id, 
                                          towards_=c_id)
            
            im_accounts.put_amounts(b_id, im_bi)
            im_accounts.put_amounts(c_id, -im_ci)
            
            c_bi = vm_i.sum() + im_bi.sum()
            
            #############
            ### DVA^i ###
            #############
            dva_ += discount(time_payoff) * lgd * np.maximum(q_i - c_bi, 0.)
                
        if zeta > maturity + delta:
            continue
        
        ##############
        ### I_zeta ###
        ##############
        i_in_I_zeta_1 = bc_min_df_time > maturity and zeta <= maturity
        i_in_I_zeta_2 = bc_min_df_time < maturity and zeta <= min(b_min_df_time, c_min_df_time + delta)
        
        i_in_I_zeta = i_in_I_zeta_1 or i_in_I_zeta_2
        
        if not i_in_I_zeta:
            continue
          
        ###########
        ### P^i ###
        ###########
        p_i = port.compute_value(zeta, from_=c_id, towards_=b_id).sum()
            
        ############
        ### VM^i ###
        ############
        vm_i = port.compute_value(zeta, from_=c_id, towards_=b_id)
        
        #################
        ### IM^{b, i} ###
        #################
        im_bi = port.compute_exposure(zeta, 
                                      risk_period=delta, 
                                      conf_level=quantile_im,
                                      from_=c_id, 
                                      towards_=b_id)
        
        #################
        ### IM^{c, i} ###
        #################
        im_ci = -port.compute_exposure(zeta, 
                                       risk_period=delta, 
                                       conf_level=quantile_im,
                                       from_=b_id, 
                                       towards_=c_id)           
    
        im_accounts.put_amounts(b_id, im_bi)
        im_accounts.put_amounts(c_id, -im_ci)
        
        ############
        ### IM^i ###
        ############
        c_i = vm_i.sum() + im_bi.sum()
        
        #############
        ### FVA^i ###
        #############
        fva_ += p_i - c_i
            
        #############
        ### KVA^i ###
        #############
        __kccr = regul_cap.compute_k_ccr(c_id, zeta, epsilon=epsilon_ead, risk_period=delta, alpha=cap_conf_level)
        __kcva = regul_cap.compute_k_cva(c_id, zeta, epsilon=epsilon_ead, risk_period=delta, alpha=cap_conf_level)
        
        kva_ += __kccr + __kcva
            
    results_globs["cva"]["sum"] += cva_
    results_globs["cva"]["sum2"] += cva_**2
    
    results_globs["dva"]["sum"] += dva_
    results_globs["dva"]["sum2"] += dva_**2
    
    tmp_mla = 0.
    tmp_mva = 0.
    tmp_kva = 0.
    if zeta <= maturity + delta:
        tmp_mla = discount_zeta * inv_pdf_zeta * (c_zeta * fva_)
        tmp_mva = discount_zeta * inv_pdf_zeta * (tilde_lambda_zeta * (-np.minimum(fva_, 0.)) - lambda_zeta * np.maximum(fva_, 0.))
        
        tmp_kva = discount_zeta * inv_pdf_zeta * k_zeta * kva_
    
    results_globs["mla"]["sum"] += tmp_mla 
    results_globs["mla"]["sum2"] += tmp_mla**2
    
    results_globs["mva"]["sum"] += tmp_mva 
    results_globs["mva"]["sum2"] += tmp_mva**2
        
    results_globs["kva"]["sum"] += tmp_kva 
    results_globs["kva"]["sum2"] += tmp_kva**2
        
toc = time.time()

# Results

In [30]:
print "Results for %d iterations (%s secs.)"%(N, toc-tic)
print

print "Used discount factor: %s"%discount
print

print "Bank index %i that belongs to the following MO copula subsets:"%b_id
for idx in b_subsets_indexes:
    print "- %s with pillars %s and intensity %s\n"%([x for x in copula.subsets[idx][0]], \
                                                     copula.pillars[idx], copula.intensities[idx])
    
print "Counterparties indexes:"
print c_ids

print "\n-----------------------------------------------------\n"

for idx in c_ids:
    print "Counterparty index %s belongs to the following MO copula subses:"%idx
    sub_indexes = copula.get_indexes_including(idx)
    for subset_idx in sub_indexes:
        print "- %s with pillars %s and intensity %s\n" % ([x for x in copula.subsets[subset_idx][0]], \
                                                           copula.pillars[subset_idx], copula.intensities[subset_idx])        
    print
    
print "\n-----------------------------------------------------\n"
    
print "Derivatives:"
for d in [swap]:
    print "- %s"%d
    
print "\nPositions:"
print port.positions

print "Confidence level used for IM: %.2f"%quantile_im

Results for 10000 iterations (937.944999933 secs.)

Used discount factor: Constant discount factor process with rate r = 0.02

Bank index 29 that belongs to the following MO copula subsets:
- [29] with pillars [3 5] and intensity [ 0.06140538  0.01950418]

- [64, 98, 99, 38, 70, 71, 104, 75, 76, 81, 78, 111, 17, 83, 120, 26, 124, 29, 31] with pillars [3 5] and intensity [ 0.0053745  0.0050889]

- [17, 26, 29, 31, 38, 50, 55, 64, 66, 70, 71, 75, 76, 78, 81, 83, 98, 99, 103, 104, 107, 111, 120, 122, 124] with pillars [3 5] and intensity [ 0.0006145  0.0086692]

- [0, 3, 4, 5, 9, 13, 17, 24, 25, 26, 27, 28, 29, 30, 31, 33, 36, 37, 38, 43, 45, 47, 50, 52, 55, 56, 57, 61, 64, 66, 70, 71, 72, 73, 75, 76, 78, 80, 81, 82, 83, 86, 87, 89, 90, 98, 99, 100, 103, 104, 107, 111, 112, 113, 114, 116, 118, 119, 120, 122, 124] with pillars [3 5] and intensity [  2.01650000e-04   9.33720000e-05]

- [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28,

In [31]:
from scipy.stats import norm

conf_level = 0.95
z_level = norm.ppf(0.5*(1+conf_level))

print "Results of the global xVA:\n"
keys = results_globs.keys()
keys.sort()

excel_results = dict()

for k in keys:
    mean_ = results_globs[k]['sum']/N
    mod_var_ = (results_globs[k]['sum2']/N - mean_**2) / (N-1.)
    half_inter = z_level*np.sqrt(mod_var_)
    
    confidence = 0. if mean_ == 0. else half_inter / mean_
    excel_results[k.upper()] = "%s \t %s"%(mean_*10000, confidence)
    
    print "The %s for the bank lies in [%f, %f]"%(k.upper(), mean_-half_inter, mean_+half_inter)

Results of the global xVA:

The CVA for the bank lies in [0.001805, 0.002063]
The DVA for the bank lies in [0.002839, 0.003332]
The KVA for the bank lies in [0.001916, 0.001964]
The MLA for the bank lies in [0.000000, 0.000000]
The MVA for the bank lies in [0.010887, 0.011213]


In [32]:
file_ = open('../Bilat Base case.txt', 'a')

file_.write("%s \n"%(b_id))

keys = excel_results.keys()
keys.sort()

for k in keys:
    file_.write("%s \t %s\n"%(k, excel_results[k]))
    
file_.write("\n")
file_.close()